# Actividad 3:
# Descubriendo estructuras ocultas: Reducción de dimensionalidad con PCA

## Objetivo
Aplicar PCA en un conjunto de datos multivariado, interpretar los resultados, visualizar la proyección en 2D, y reflexionar sobre el impacto de la reducción de dimensionalidad en el análisis exploratorio de datos y la preparación de modelos predictivos.

**Datasets utilizados:**  
`Wine`

---

### Estructura del Notebook:
1. Metodología.
2. Configuración del entorno.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

## 1. Metodología

### Flujo de trabajo

1. **Carga, preprocesamiento de datos y aplicación de PCA exploratorio:**
    - Se carga el dataset **Wine** y se escala.
    - Se aplica PCA para obtener la varianza acumulada y ver como varía con cada componente.

2. **Aplicación de KNN:**
    - Se preparan los datos, haciendo train_test_split a partir de X e y original y escalando sobre los splits.
    - Uso de optuna para encontrar los mejores hiperparámetros para el modelo KNeighborsClassifier, se usa para evaluar con y sin PCA.
    - Entrenamiento del modelo con los mejores hiperparámetros con y sin PCA.

3. **Visualización e interpretación:**
    - Tabla de evolución de la varianza acumulada por componente.
    - Gráfico de varianza acumulada por componente y gráfico de PCA con 2 componentes.
    - Comparación de métricas de modelos KNeighborsClassifier con y sin PCA.
    - Matriz de confusión de los modelos con y sin PCA.

---

# 2. Configuración del entorno

--- 

In [ ]:
# Importaciones de librerías de terceros
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna

# Importaciones de scikit-learn
from sklearn.datasets import load_wine
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# Configuraciones de estilo
sns.set_theme(style="whitegrid")
pd.set_option("display.max_columns", None)

# Desactiva el logging de Optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)  # Solo muestra warnings y errores

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

---

**Bloque 1:** Carga, preprocesamiento de datos y aplicación de PCA exploratorio.

- **`carga_y_preprocesamiento()`** 
Carga el dataset Wine, realiza escalado y prepara los datos para el análisis.

- **`aplicar_pca()`** 
Aplica PCA para obtener componentes principales y la varianza explicada acumulada.
---

`Justificación del uso de None en n_componentes`

Para entender cuánta información contiene cada componente principal, primero calculamos todas las componentes posibles del dataset (en este caso, 13 características originales). Esto permite obtener la varianza explicada y acumulada completa, facilitando la selección informada de un número reducido de componentes que retengan la mayor parte de la información (normalmente, un umbral del 90% o más). Esta etapa es fundamental para justificar posteriormente cuántos componentes usar en el modelo.

---

In [ ]:
def carga_y_preprocesamiento():
    """
    Carga y preprocesa el dataset Wine de scikit-learn.

    - Carga los datos y la variable objetivo.
    - Muestra la forma y distribución de clases.
    - Verifica la ausencia de valores nulos.
    - Escala las características usando StandardScaler.

    Returns:
        X_scaled_df (pd.DataFrame): Datos escalados con nombres de columnas.
        X (pd.DataFrame): Datos originales sin escalar.
        y (pd.Series): Variable objetivo (clases).
    """
    # Cargar el dataset Wine de scikit-learn
    wine = load_wine()
    X = pd.DataFrame(wine.data, columns=wine.feature_names)
    y = pd.Series(wine.target, name="target") # La variable objetivo en Wine se llama 'target'

    # Vista general e información del dataset
    print("Forma del dataset (X):", X.shape)
    print("Forma del dataset (y):", y.shape)
    X.info()

    # Distribución de clases de la variable objetivo
    distribucion_de_clases_target = y.value_counts()
    print("\nDistribución de clases de la variable objetivo:")
    print(distribucion_de_clases_target)

    # Revisar si no hay nulos antes de escalar
    if X.isnull().sum().any():
        print("Advertencia: Hay valores nulos en los datos después del preprocesamiento.")
    else:
        print("\nNo hay valores nulos en los datos después del preprocesamiento.")

    # Escalar cuando tenemos todo limpio
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Opcional: convertir X_scaled de nuevo a DataFrame para mantener los nombres de las columnas
    X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

    return X_scaled_df, X, y


def aplicar_pca(X_scaled):
    """
    Aplica Análisis de Componentes Principales (PCA) al dataset escalado.

    - Calcula todas las componentes para obtener varianza explicada y acumulada.
    - Facilita análisis exploratorio para selección del número óptimo de componentes.

    Args:
        X_scaled (np.ndarray o pd.DataFrame): Datos escalados.

    Returns:
        X_pca (np.ndarray): Datos transformados por PCA.
        varianza_explicada (np.ndarray): Varianza explicada por cada componente.
        varianza_acumulada (np.ndarray): Varianza explicada acumulada.
    """
    # Reducir a 2 componentes para visualización
    pca = PCA(n_components=None)  # para calcular todas y ver la varianza acumulada
    X_pca = pca.fit_transform(X_scaled)

    # Varianza explicada
    varianza_explicada = pca.explained_variance_ratio_
    varianza_acumulada = np.cumsum(varianza_explicada)

    return X_pca, varianza_explicada, varianza_acumulada

**Bloque 2:** Aplicación de KNN con y sin PCA.

- **`preparar_datos_modelo()`** 
Divide y preprocesa los datos para entrenamiento y prueba, opcionalmente aplicando PCA.

- **`buscar_hiperparametros_optimos()`** 
Optimiza los hiperparámetros del modelo KNN usando Optuna y validación cruzada estratificada.

- **`entrenar_y_evaluar_modelo()`** 
Entrena el modelo KNN con parámetros dados y evalúa su rendimiento con métricas clave.

---

`Justificación de escalar nuevamente los datos`

Los datos se escalan primero sobre el dataset completo para un análisis exploratorio global y aplicación inicial de PCA. Luego, tras dividir los datos en entrenamiento y prueba, se realiza un escalado independiente sobre el conjunto de entrenamiento y se aplica la misma transformación al conjunto de prueba. Esto es clave para evitar filtrado de información del test set en el entrenamiento (data leakage), y para simular condiciones reales en donde el modelo no debe conocer datos futuros.

---

`Justificación de uso de 8 componentes en PCA`

Se seleccionaron 8 componentes principales en el análisis PCA porque estos explican aproximadamente el 92.02% de la varianza acumulada de los datos originales. Este umbral representa un equilibrio adecuado entre reducción de dimensionalidad y retención de información:

- Captura la mayoría de la estructura de los datos, evitando pérdida significativa de información relevante.
- Reduce el ruido y la redundancia, ya que se eliminan componentes que aportan muy poca varianza.
- Mejora la eficiencia computacional sin comprometer gravemente el rendimiento predictivo del modelo.
- Este umbral del 92% es comúnmente aceptado como suficientemente alto para preservar las relaciones fundamentales en los datos, evitando el sobreajuste que puede darse con demasiadas variables.
- Por tanto, 8 componentes representan un punto óptimo para mantener la calidad del modelo y facilitar su interpretación.

---

`Justificación del uso de KNN con y sin PCA`

El algoritmo K-Nearest Neighbors (KNN) es sensible a la dimensionalidad y a la escala de los datos, lo cual lo convierte en una excelente opción para evaluar el impacto de técnicas de reducción dimensional como PCA (Análisis de Componentes Principales).

KNN sin PCA:
- Evaluar el modelo directamente sobre los datos originales permite observar su desempeño con toda la información disponible. Esto actúa como línea base ("baseline") para comparar contra cualquier técnica de reducción o transformación.

KNN con PCA:
- Al reducir la dimensionalidad con PCA:
- Se elimina ruido o redundancia en los datos, lo que puede ayudar a mejorar la generalización.
- Se reduce el costo computacional al trabajar con menos dimensiones.
- Se busca mejorar el rendimiento o mantenerlo con menor complejidad.
- Es útil cuando los datos tienen muchas variables correlacionadas (como es el caso del dataset Wine).

Comparar ambos escenarios permite:
- Evaluar si la reducción de dimensiones sacrifica o mejora el rendimiento.
- Observar el trade-off entre precisión y eficiencia.
- Identificar si el modelo depende de características específicas que podrían perderse al aplicar PCA.

---

In [ ]:
def preparar_datos_modelo(X, y, usar_pca=False, n_componentes_pca=8, test_size=0.3, random_state=42):
    """
    Prepara los datos para el entrenamiento del modelo.

    - Divide el dataset en entrenamiento y prueba con estratificación para conservar distribución de clases.
    - Escala los datos usando StandardScaler ajustado solo sobre entrenamiento.
    - Aplica PCA si se especifica con el número de componentes deseado.

    Args:
        X (pd.DataFrame o np.ndarray): Datos originales.
        y (pd.Series o np.ndarray): Variable objetivo.
        usar_pca (bool): Indica si se aplica PCA.
        n_componentes_pca (int): Número de componentes para PCA, calculado con la varianza acumulada.
        test_size (float): Proporción de test set.
        random_state (int): Semilla para reproducibilidad.

    Returns:
        Tuple: Datos divididos y preprocesados para entrenamiento y prueba.
        Si usar_pca es True: (X_train_pca, X_test_pca, y_train, y_test)
        Si usar_pca es False: (X_train_scaled, X_test_scaled, y_train, y_test)
    """
    # Split con estratificación
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    # Escalado
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    if usar_pca:
        pca = PCA(n_components=n_componentes_pca)
        X_train_pca = pca.fit_transform(X_train_scaled)
        X_test_pca = pca.transform(X_test_scaled)
        return X_train_pca, X_test_pca, y_train, y_test
    else:
        return X_train_scaled, X_test_scaled, y_train, y_test

def buscar_hiperparametros_optimos(X_train, y_train, n_trials=30):
    """
    Optimiza hiperparámetros de KNN usando Optuna con validación cruzada estratificada.

    - Busca número óptimo de vecinos, tipo de peso y métrica de distancia.
    - Utiliza StratifiedKFold para preservar distribución de clases en folds.
    - Maximiza accuracy promedio en validación cruzada.

    Args:
        X_train (np.ndarray): Datos de entrenamiento.
        y_train (np.ndarray): Etiquetas de entrenamiento.
        n_trials (int): Número de pruebas de hiperparámetros.

    Returns:
        dict: Mejores hiperparámetros encontrados por Optuna.
    """
    def objective(trial):
        n_neighbors = trial.suggest_int("n_neighbors", 1, 30)
        weights = trial.suggest_categorical("weights", ["uniform", "distance"])
        metric = trial.suggest_categorical("metric", ["euclidean", "manhattan", "chebyshev"])

        knn = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, metric=metric)
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

        score = cross_val_score(knn, X_train, y_train, cv=skf, scoring="accuracy").mean()
        return score

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)
    return study.best_params


def entrenar_y_evaluar_modelo(X_train, X_test, y_train, y_test, params):
    """
    Entrena el modelo KNN con hiperparámetros dados y evalúa su desempeño.

    - Ajusta el modelo con datos de entrenamiento.
    - Predice sobre datos de prueba.
    - Calcula métricas de clasificación multiclasica (accuracy, precision, recall, f1).
    - Calcula la matriz de confusión.

    Args:
        X_train (np.ndarray): Datos de entrenamiento.
        X_test (np.ndarray): Datos de prueba.
        y_train (np.ndarray): Etiquetas de entrenamiento.
        y_test (np.ndarray): Etiquetas de prueba.
        params (dict): Hiperparámetros para KNeighborsClassifier.

    Returns:
        modelo: Modelo entrenado.
        metrics (dict): Diccionario con métricas y matriz de confusión.
    """
    # Entrenamiento
    knn = KNeighborsClassifier(**params)
    knn.fit(X_train, y_train)

    # Predicción
    y_pred = knn.predict(X_test)

    # Métricas
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, average="macro", zero_division=0),
        "recall": recall_score(y_test, y_pred, average="macro", zero_division=0),
        "f1_score": f1_score(y_test, y_pred, average="macro", zero_division=0),
        "confusion_matrix": confusion_matrix(y_test, y_pred)
    }

    return knn, metrics

**Bloque 3:** Visualización de resultados.

- **`tabla_varianza_acumulada()`** 
Muestra una tabla con la varianza explicada y acumulada por cada componente PCA.

- **`graficar_pca_combinado()`** 
Grafica la varianza acumulada y la proyección de los datos en los dos primeros componentes PCA.

- **`mostrar_comparacion_metricas()`** 
Presenta una tabla comparativa de métricas entre modelos con y sin PCA.

- **`mostrar_matrices_confusion()`** 
Visualiza matrices de confusión para los modelos con y sin PCA para análisis detallado.

---

In [ ]:
def tabla_varianza_acumulada(varianza_explicada, varianza_acumulada):
    """
    Muestra una tabla con la varianza explicada y acumulada por cada componente PCA.

    Args:
        varianza_explicada (np.ndarray): Varianza explicada individual por componente.
        varianza_acumulada (np.ndarray): Varianza explicada acumulada.
    """
    df_varianza = pd.DataFrame({
        'Componente': range(1, len(varianza_explicada) + 1),
        'Varianza Explicada (%)': varianza_explicada * 100,
        'Varianza Acumulada (%)': varianza_acumulada * 100
    })

    display(df_varianza.round(2))

def graficar_pca_combinado(varianza_acumulada, X_pca, y):
    """
    Grafica la varianza explicada acumulada y la proyección de los dos primeros componentes PCA.

    - Primer gráfico: línea con varianza acumulada para selección de componentes.
    - Segundo gráfico: scatter plot de PC1 vs PC2 con colores según la clase.

    Args:
        varianza_acumulada (np.ndarray): Varianza acumulada para componentes PCA.
        X_pca (np.ndarray): Datos transformados por PCA.
        y (pd.Series o np.ndarray): Etiquetas de clase para colorear.
    """
    fig, axs = plt.subplots(1, 2, figsize=(14, 5))

    # Gráfico de varianza acumulada
    axs[0].plot(range(1, len(varianza_acumulada) + 1), varianza_acumulada, marker='o', linestyle='--')
    axs[0].set_xlabel('Número de Componentes')
    axs[0].set_ylabel('Varianza Explicada Acumulada')
    axs[0].set_title('Varianza Explicada Acumulada por PCA')
    axs[0].grid(True)

    # Gráfico de los 2 primeros componentes PCA con etiquetas
    sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=y, palette='Set1', ax=axs[1])
    axs[1].set_xlabel("PC1")
    axs[1].set_ylabel("PC2")
    axs[1].set_title("Proyección PCA (2 Componentes)")
    axs[1].legend(title='Condition')
    axs[1].grid(True)

    plt.tight_layout()
    plt.show()

def mostrar_comparacion_metricas(metrics_sin_pca, metrics_con_pca):
    """
    Muestra una tabla comparativa con métricas de modelos entrenados sin y con PCA.

    Args:
        metrics_sin_pca (dict): Métricas del modelo sin PCA.
        metrics_con_pca (dict): Métricas del modelo con PCA.
    """
    # Excluir matriz de confusión
    dict_sin_pca = {k: v for k, v in metrics_sin_pca.items() if k != "confusion_matrix"}
    dict_con_pca = {k: v for k, v in metrics_con_pca.items() if k != "confusion_matrix"}

    df_comparativa = pd.DataFrame({
        "Sin PCA": dict_sin_pca,
        "Con PCA": dict_con_pca
    })

    print("\nComparación de métricas KNN:")
    display(df_comparativa.style.format("{:.4f}"))

def mostrar_matrices_confusion(metrics_sin_pca, metrics_con_pca, clases=None):
    """
    Muestra matrices de confusión lado a lado para modelos sin y con PCA.

    Args:
        metrics_sin_pca (dict): Métricas que incluyen matriz de confusión del modelo sin PCA.
        metrics_con_pca (dict): Métricas que incluyen matriz de confusión del modelo con PCA.
        clases (list, opcional): Etiquetas para las clases a mostrar en la matriz.
    """
    cm_sin = metrics_sin_pca["confusion_matrix"]
    cm_con = metrics_con_pca["confusion_matrix"]

    if clases is None:
        clases = ['0', '1', '2']

    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    sns.heatmap(cm_sin, annot=True, fmt='d', cmap='Blues', ax=axes[0], cbar=False,
                xticklabels=clases, yticklabels=clases)
    axes[0].set_title('Matriz de Confusión SIN PCA')
    axes[0].set_xlabel('Predicción')
    axes[0].set_ylabel('Verdadero')

    sns.heatmap(cm_con, annot=True, fmt='d', cmap='Greens', ax=axes[1], cbar=False,
                xticklabels=clases, yticklabels=clases)
    axes[1].set_title('Matriz de Confusión CON PCA')
    axes[1].set_xlabel('Predicción')
    axes[1].set_ylabel('Verdadero')

    plt.tight_layout()
    plt.show()

**Bloque 4:** Función de ejecución.

- **`main()`** 
Ejecuta el flujo completo del proyecto, desde la carga de datos hasta la evaluación y visualización de resultados.

---

In [ ]:
def main():
    """
    Función principal que orquesta la carga, preprocesamiento, análisis PCA, 
    optimización de hiperparámetros, entrenamiento, evaluación y visualización 
    de modelos KNN con y sin reducción dimensional mediante PCA sobre el dataset Wine.
    """
    print("=== Carga y preprocesamiento de datos ===\n")
    X_scaled, X, y = carga_y_preprocesamiento()
    
    print("\n=== Aplicando PCA para análisis exploratorio ===")
    X_pca, varianza_explicada, varianza_acumulada = aplicar_pca(X_scaled)
    
    print("\n=== Preparando datos para modelos KNN (sin y con PCA) ===")
    X_train, X_test, y_train, y_test = preparar_datos_modelo(X, y, usar_pca=False)
    X_train_pca, X_test_pca, y_train_pca, y_test_pca = preparar_datos_modelo(X, y, usar_pca=True)
    
    print("\n=== Buscando hiperparámetros óptimos con Optuna ===")
    mejores_params_sin_pca = buscar_hiperparametros_optimos(X_train, y_train)
    mejores_params_con_pca = buscar_hiperparametros_optimos(X_train_pca, y_train_pca)
    
    print("\n=== Entrenando y evaluando los modelos ===")
    modelo_sin_pca, metrics_sin_pca = entrenar_y_evaluar_modelo(
        X_train, X_test, y_train, y_test, mejores_params_sin_pca
    )
    modelo_con_pca, metrics_con_pca = entrenar_y_evaluar_modelo(
        X_train_pca, X_test_pca, y_train_pca, y_test_pca, mejores_params_con_pca
    )
    
    print("\n=== Resultados y visualización ===")
    tabla_varianza_acumulada(varianza_explicada, varianza_acumulada)
    graficar_pca_combinado(varianza_acumulada, X_pca, y)
    
    mostrar_comparacion_metricas(metrics_sin_pca, metrics_con_pca)
    mostrar_matrices_confusion(metrics_sin_pca, metrics_con_pca)

# 4. Visualización de resultados

Se muestran los resultados obtenidos a partir de la ejecución de la funcion **main()**.

---

In [ ]:
if __name__ == "__main__":
    main()

# 5. Análisis de los resultados y reflexiones finales

---

### ¿Cuánta información conserva el nuevo espacio?

Al aplicar el Análisis de Componentes Principales (PCA), observamos que las primeras ocho componentes principales son capaces de capturar el 92.02% de la varianza total del conjunto de datos original. Este porcentaje sugiere que, aunque se ha producido una compresión considerable de la dimensionalidad, se mantiene gran parte de la estructura estadística esencial de los datos.

Específicamente, las dos primeras componentes ya explican más del 55% de la varianza, lo que indica que las dimensiones originales estaban altamente correlacionadas y que una representación más compacta era no solo posible, sino también efectiva. Este tipo de reducción facilita tanto la visualización como el análisis, sin que ello implique una pérdida significativa de información relevante para las tareas de clasificación.

### ¿Se observan agrupamientos o patrones más claros?

La proyección de los datos en el nuevo espacio bidimensional generado por las dos primeras componentes principales revela estructuras latentes que no eran evidentes en el espacio original. En este plano reducido, emergen tres agrupamientos claramente diferenciables, los cuales se alinean con las clases originales 0, 1 y 2.

Esta separación no es trivial: el hecho de que clases diferentes se proyecten en regiones distintas del espacio PCA sugiere que existen relaciones subyacentes entre las variables originales que son eficientemente capturadas por la transformación. Más allá de ser una herramienta de reducción, PCA actúa aquí como un método exploratorio poderoso, evidenciando la estructura interna del conjunto de datos y mostrando que las clases están razonablemente bien separadas incluso sin supervisión directa.

---

### Análisis del rendimiento del modelo KNN

> Nota: Para efectos de este trabajo, no se discutira sobre el sobreajuste o baja generalización que puedan presentar los modelos, si no que se usaran como comparación para ver como actúa con datos con y sin procesado de PCA.

Se entrenaron dos versiones del modelo KNN: una con los datos originales (sin aplicar PCA) y otra sobre los datos transformados mediante PCA (usando las primeras 8 componentes). En una ejecución controlada con random_state=42, ambos modelos reportaron idénticas métricas de desempeño:

| Métrica   | Sin PCA | Con PCA |
|-----------|---------|---------|
| Accuracy  | 0.9815  | 0.9815  |
| Precision | 0.9792  | 0.9792  |
| Recall    | 0.9841  | 0.9841  |
| F1-Score  | 0.9811  | 0.9811  |


Este comportamiento no implica que PCA sea irrelevante para el modelo, sino que en este escenario específico —con la semilla fija— el proceso de validación cruzada encontró combinaciones de entrenamiento y prueba donde ambos enfoques resultan igualmente efectivos.

Sin embargo, al remover la semilla (como en el código final) y permitir la aleatoriedad natural del ShuffleSplit, se observaron variaciones sutiles en las métricas entre ejecuciones. En general, el modelo entrenado sobre los datos reducidos por PCA tiende a presentar un rendimiento ligeramente superior, especialmente en términos de precisión y F1-score, aunque las diferencias no siempre son estadísticamente significativas. Esto sugiere que la transformación PCA puede mejorar la estabilidad y generalización del modelo en distintos subconjuntos del conjunto de datos. Ejamplo de ejecución sin semilla específica:

| Métrica   | Sin PCA | Con PCA |
|-----------|---------|---------|
| Accuracy  | 0.9630  | 0.9815  |
| Precision | 0.9616  | 0.9792  |
| Recall    | 0.9683  | 0.9841  |
| F1-Score  | 0.9636  | 0.9811  |

La matriz de confusión correspondiente revela una clasificación casi perfecta: un único error en el cual un dato de clase 1 fue clasificado como clase 2. El resto de las instancias fueron correctamente etiquetadas. Esta precisión sugiere que el modelo capta eficazmente las fronteras de decisión, y que los errores se limitan a observaciones fronterizas o ambiguas.

---

### Conclusión

Este análisis demostró que la reducción de dimensionalidad mediante Análisis de Componentes Principales (PCA) es una técnica altamente efectiva para simplificar el espacio de características sin comprometer el rendimiento predictivo del modelo.

La selección de 8 componentes principales permitió conservar un 92.02% de la varianza total, lo que implica una pérdida mínima de información. Además, la visualización en el espacio de los primeros dos componentes mostró una clara separación entre clases, reflejando una estructura subyacente bien definida en los datos originales.

Respecto al modelo K-Nearest Neighbors (KNN), se observó un rendimiento prácticamente idéntico con y sin PCA cuando se fijó la semilla aleatoria (random_state=42). No obstante, al permitir la aleatoriedad natural, los modelos entrenados sobre los datos reducidos por PCA mostraron un rendimiento ligeramente superior en métricas clave, lo cual sugiere que la reducción de dimensionalidad no solo facilita la visualización y simplificación del problema, sino que también puede aumentar la robustez y capacidad de generalización del modelo.

Por último, el análisis de la matriz de confusión indicó un desempeño casi perfecto, con tan solo un único error de clasificación en ambos casos. Este resultado, junto con las métricas obtenidas, confirma que el uso combinado de PCA y KNN puede ser una estrategia eficiente y confiable tanto para compresión de datos como para clasificación precisa en conjuntos complejos.

---